# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name: Emmanuel Macario
###### Student Number: 831659
###### Email: <macarioe@student.unimelb.edu.au>
###### Python version: 3.6.0

# Libraries Used

In [1]:
# Import useful libraries
from collections import defaultdict
import pandas as pd
import numpy as np

# Preprocess Function

In [2]:

def preprocess(filename):
    """
    Opens a data file in csv format, and transforms it 
    into a usable format, in the form of a pandas DataFrame.
    """
    
    # Read csv data into a dataframe. Assign
    # an integer value to each attribute in
    # the data.
    df = pd.read_csv(filename, header=None)
        
    
    # Partition the data into a list of instances, and 
    # a list of class labels for those instances
    instance_list = df.iloc[:,:-1]
    class_list = df.iloc[:,-1]
    data_set = instance_list, class_list
    
    
    return data_set




def proportion_missing_values(instance_list):
    """
    Calculates and returns the proportion of 
    instances that contain at least one missing value
    in the entire dataset. Used for personal analysis 
    of the data.
    """
    
    return len(instance_list[instance_list.values == '?'])/len(instance_list)
    

# Train Supervised Function

In [3]:

def train_supervised(instance_list, class_list):
    """
    Builds a supervised Naive Bays model, given a 
    preprocessed set of training data, by calculating 
    counts from the training data.
    
    Takes as input a training set of data, consisting
    of a list of instances, and a list of class labels 
    corresponding to those instances.
    """
    
    # The supervised Naive Bays model
    supervised_model = {}
    
    # For every instance in the training
    # data, update the model
    for i in range(len(instance_list)):
        
        # Get the instance
        instance = instance_list.iloc[i,:]
        
        # Get the associated class label
        class_label = class_list[i]
        
        
        # If the class label is not already in
        # the model, create a nested set of
        # dictionaries for the class.
        if class_label not in supervised_model:
            
            supervised_model[class_label] = {}
            
            # For each attribute, initialise
            # a new default dictionary.
            for attr in instance_list.columns:
                supervised_model[class_label][attr] = defaultdict(int)
                
        
        # Then, for every attribute in the instance,
        # get its corresponding value and update
        # the model.
        for attr in instance_list.columns:
            
            attr_value = instance[attr]
            
            # For missing values in a training instance, it is
            # possible to simply have them not contribute to the
            # attribute-value counts.
            if attr_value == '?':
                continue
                
            supervised_model[class_label][attr][attr_value] += 1
    
    
    return supervised_model


# Predict Supervised Function

In [4]:

def predict_supervised(supervised_model, instance_list):
    """
    Predicts the class labels for a set of test
    instances, based on a supervised Naive Bayes
    model. Implements epsilon probabilistic smoothing.
    """
    
    # Epsilon value for epsilon-smoothing
    EPSILON = np.finfo(float).eps
    
    
    # The list of predictions for each instance
    prediction_list = []
    
    
    # Get the class frequencies to avoid redundant calculations
    class_freqs = get_class_freqs(supervised_model)
    
    
    # Calculate the total number of instances,
    total_instances = sum(class_freqs.values())
    
    
    # For each instance in the test set, predict 
    # its class based on the supervised model.
    for instance in instance_list.values:
        
        max_prob = 0.0
        predicted_class = ""
        
        # We obtain the probability of an instance being a 
        # certain class, P(C|X), for each possible class.
        for class_label in class_freqs:
            
            # Firstly, get the prior probability of a class, P(Cj)
            prob = class_freqs[class_label]/total_instances
            
            
            # Now, multiply the prior probability of the class
            # by the posterior probability of each of the instance's 
            # attribute values, given the class, P(Xi|Cj).
            for attr in supervised_model[class_label]:
                
                # Get the attribute value
                attr_value = instance[attr]
                
                # If an attribute value is missing for the training
                # instance, simply have it not count towards the probability
                # estimates
                if attr_value == '?':
                    continue
                
                # When accessing values, if value is 0, replace with epsilon
                attr_prob = supervised_model[class_label][attr].get(attr_value, EPSILON) / class_freqs[class_label]
                
                # Update the probability estimate
                prob *= attr_prob
                
            
            # If the probability is the highest seen
            # thus far, set the predicted class to the
            # class label.
            if prob > max_prob:
                max_prob = prob
                predicted_class = class_label
            
        
        # Add the class label with the highest
        # corresponding probability to the list
        # of predictions.
        prediction_list.append(predicted_class)
        
        
    return prediction_list




def get_class_freqs(supervised_model):
    """
    Helper function that returns a dictionary containing 
    frequency counts of all the class labels in the training data.
    """
    
    class_freqs = defaultdict(int)
    
    for class_label in supervised_model:
        class_freqs[class_label] = sum(supervised_model[class_label][0].values())
        
    return class_freqs


# Evaluate Supervised Function

In [5]:

def evaluate_supervised(prediction_list, class_list):
    """
    Evaluates a set of predictions, in a supervised
    context. Uses accuracy as the primary method of
    evaluation.
    """
    
    # Validation checking
    assert(len(prediction_list) == len(class_list))
    
    # Calculate and return the accuracy of the model
    return (prediction_list == class_list).value_counts().loc[True] / len(prediction_list)


# Train Unsupervised Function

In [6]:

def train_unsupervised(instance_list, class_labels, class_distributions=None):
    """
    Builds a weak unsupervised Naive Bayes model,
    from a given set of unlabelled training data,
    and a unique set of possible class labels. This
    function is called by predict_unsupervised() to
    iteratively update the random class distributions.
    """
    
    # If the random class distributions have not been
    # created, then for each training instance, create
    # a non-uniform (random) class distribution.
    if class_distributions is None:
        
        random_class_distributions = []

        for instance in instance_list.values:

            random_distribution = np.abs(np.random.randn(1, len(class_labels))[0])
            norm_random_distribution = random_distribution / sum(random_distribution)
            random_class_distributions.append(norm_random_distribution)

        # Store all random class distributions in a dataframe
        class_distributions = pd.DataFrame(random_class_distributions, 
                                           columns=class_labels)
        
        
        
    # Initialise the unsupervised Naive Bays model, 
    # which is just a set of nested dictionaries
    unsupervised_model = {c : {a : defaultdict(float) for a in instance_list.columns} for c in class_labels}
    
    
    # For every instance in the training set, update
    # fractional counts in the unsupervised model
    for i in range(len(instance_list)):
        
        for class_label in class_labels:
            
            for attr in instance_list.columns:
            
                attr_value = instance_list[attr][i]
                
                # If an attribute value is missing,
                # simply have it not contribute to counts
                if attr_value == '?':
                    continue
                
                unsupervised_model[class_label][attr][attr_value] += class_distributions[class_label][i]
                
    
    return class_distributions, unsupervised_model


# Predict Unsupervised Function

In [7]:

def predict_unsupervised(class_distributions, unsupervised_model, instance_list, iterations=5):
    """
    Predicts the class distribution for a set of
    instances, based on a trained model. Delegates
    the task of updating class distributions in a
    single iteration to the train_unsupervised() function.
    Once the 'final' class distributions have been calculated,
    it classifies the instances based on the distributions.
    """
        
    # For each iteration, update the class distribution for each
    # instance, in an attempt to make them more reliable than
    # the initial random distribution.
    for iteration in range(iterations):
        
        # Calculate the frequency of the classes 
        # in the random class distribution
        class_freqs = get_class_freqs2(class_distributions)
         
        
        # Calculate the total number of instances
        total_instances = int(sum(class_freqs.values()) + 0.5)
    
        
        for i in range(len(instance_list)):

            instance = instance_list.iloc[i,:]
    
            for class_label in class_freqs:

                # Firstly, get the prior probability of a class
                prob = class_freqs[class_label] / total_instances

                # Now, multiply the prior probability of the class
                # by the posterior probability of each of the instance's 
                # attribute values, given the class.
                for attr in unsupervised_model[class_label]:
                    
                    attr_value = instance[attr]
                    
                    # Simply not have missing values count
                    # towards probability estimates.
                    if attr_value == '?':
                        continue
                    
                    prob *= (unsupervised_model[class_label][attr][attr_value] / class_freqs[class_label])


                # Update the class distribution for the instance
                class_distributions[class_label][i] = prob


        # Now, normalise the class distributions so that the 
        # probabilities add to 1 for every single distribution.
        class_distributions = class_distributions.div(class_distributions.sum(axis=1), axis=0)
        

        # Finally, update the unsupervised model based on the updated class distributions
        class_distributions, unsupervised_model = train_unsupervised(instance_list, 
                                                                     class_distributions.columns, 
                                                                     class_distributions)
        
    
    # The list of class predictions for the instances
    prediction_list = []
    
    
    # Assign the class with the highest probability in
    # the final distribution as the predicted class, and
    # add it to the list of predictions.
    for distribution in class_distributions.values:
        
        predicted_class = class_distributions.columns[np.argmax(distribution)]
        
        prediction_list.append(predicted_class)
            
            
    return prediction_list




def get_class_freqs2(class_distributions):
    """
    Helper function that returns a dictionary
    containing classes as keys, and the total
    sum of the fractional values for each class
    as values.
    """
    
    class_freqs = defaultdict(float)
    
    for class_label in class_distributions.columns:
        class_freqs[class_label] = class_distributions[class_label].sum()
        
    return class_freqs


# Evaluate Unsupervised Function

In [8]:

def evaluate_unsupervised(prediction_list, class_list):
    """
    Evaluates a set of predictions, in an
    unsupervised manner. Takes as input a set of 
    final class distributions for each instance, and 
    the actual class for each respective instance.
    """
    
    # Validation checking
    assert(len(prediction_list) == len(class_list))
    
    
    # Initialise and fill in the confusion matrix
    confusion_matrix = {predicted_class : defaultdict(int) for predicted_class in set(prediction_list)}
    
    for i in range(len(prediction_list)):
        predicted_class = prediction_list[i]
        actual_class = class_list[i]
        confusion_matrix[predicted_class][actual_class] += 1
    
    
    # Print the confusion matrix
    print("Confusion Matrix\n================")
    for predicted_class in confusion_matrix:
        print("{:20s}:".format(predicted_class) + "   {0}".format(confusion_matrix[predicted_class]))
    
    
    # Calculate the total number of 'correct' predictions. We
    # take the max value from each column in the confusion matrix,
    # and add it to the total number of correct predictions.
    total_correct = 0
    for predicted_class in confusion_matrix:
        total_correct += max(confusion_matrix[predicted_class].values())
    
    
    # Finally, calculate accuracy of the classifier
    accuracy = total_correct / len(prediction_list)
    
    
    return accuracy


# Driver Functions
These functions are drivers for both supervised and unsupervised Naive Bayes classifiers

In [16]:
# Driver for supervised Naive Bayes classification

def supervised_naive_bayes(filename):
    """
    Creates a supervised Naive Bayes model
    given a set of input data, classifies
    the instances based on the probabilistic 
    model, then prints the accuracy of the 
    classifier.
    """
    
    instance_list, class_list = preprocess(filename)
    supervised_model = train_supervised(instance_list, class_list)
    prediction_list = predict_supervised(supervised_model, instance_list)
    accuracy = evaluate_supervised(prediction_list, class_list)
    print("Accuracy: {:.10f}".format(accuracy))
    
    
# Driver for unsupervised Naive Bayes classification
    
def unsupervised_naive_bayes(filename):
    """
    Creates a weak unsupervised Naive Bayes model
    given a set of training instances, classifies
    the instances by iteratively updating an initial
    set of random class distributions, then
    prints the accuracy of the classifier.
    """
    
    instance_list, class_list = preprocess(filename)
    class_labels = class_list.unique()
    class_distributions, unsupervised_model = train_unsupervised(instance_list, class_labels)
    prediction_list = predict_unsupervised(class_distributions, unsupervised_model, instance_list)
    accuracy = evaluate_unsupervised(prediction_list, class_list)
    print("Accuracy: {:.10f}".format(accuracy))
    
    
# Filename constants used for easy file access
CSV1 = 'breast-cancer-dos.csv'
CSV2 = 'car-dos.csv'
CSV3 = 'hypothyroid-dos.csv'
CSV4 = 'mushroom-dos.csv'


# Testing driver functions
filename = CSV4

unsupervised_naive_bayes(filename)


Confusion Matrix
e                   :   defaultdict(<class 'int'>, {'e': 1896, 'p': 1670})
p                   :   defaultdict(<class 'int'>, {'p': 2246, 'e': 2312})
Accuracy: 0.5179714426


# Discussion


### Question 1
Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.


**Answer:** _The unsupervised classifier works well on the datasets that have both a small number of distinct classes, and a small number of distinct values for their attributes. Furthermore, the classifier achieves lower variance in accuracy for datasets containing instances that are partitioned non-uniformly among classes._

_For example, the classifier shows high variance in accuracy for the mushroom data with a mean of 0.813, a low of 0.518 and a high of 0.895. This variance could be the result of many attributes having a high number of unique values, with a max of 12 unique values for attribute 8, and a mean of 5.32 unique values per attribute. Compounded with the fact that the training data is partitioned pretty evenly among two classes (4208 instances of 'e', and 3916 of 'p'), this could have made it difficult for the classifier to converge to a particular set of predictions, hence the randomness of the results._

_Juxtaposed, consistent results are shown for the hypothyroid and breast cancer datasets, with mean accuracies of 0.952 and 0.703 respectively. 17 of 18 attributes in the hypothyroid data have only 2 possible unique values. A possible further reason for low variance could be that the distribution of instances among classes is highly non-uniform for both datasets. This could possibly inflate accuracy, since accuracy is derived by taking the maximum value from each column in the confusion matrix. If most instances are of one class, then these values tend to be large._


**Note to assessor: corresponding accuracies for supervised NB are detailed in the response to question two.**

### Question 2
When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.


**Answer:** _The accuracies of the supervised classifier across the datasets can be seen below:_

| Dataset        | Accuracy      |
| -------------  |:-------------:|
| _breast-cancer_| 0.755         | 
| _car_          | 0.874         |
| _hypothyroid_  | 0.952         |
| _mushroom_     | 0.997         |


_Since accuracy measures the ratio of correctly classified instances across all classes, this means that if one class occurs more than others, then the resulting accuracy is clearly dominated by the accuracy of the dominating class. This is the case for the hypothyroid dataset, since the classifier predicts 'negative' for all instances, when the training set is primarily dominated by 'negative' instances._

_A surprising result was the accuracy of 0.997 for the mushroom data. Such a high accuracy was not expected since 30.5% of instances contained at least one missing value. The reason a high accuracy was received is due to there being a large number of training instances containing a broad distribution of attribute value-class pairs, and because Naive Bayes gracefully degrades when one or more predictor variables are missing or not observed. The latter is due to the implementation of epsilon smoothing and simply not having missing values contribute to frequency counts for their respective features._

_Compared to the other results, the accuracy of 0.755 for the breast cancer data was relatively low. Unlike the other datasets, the training set was relatively small, perhaps leading to a probabilistic model that did not cover a wide spectrum of attribute value-class pairs. This could have possibly led to the moderately low prediction accuracy._

